In [109]:
import pandas as pd
import itertools # for confusion matrix 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from bs4 import BeautifulSoup
import re
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics # for accuracy score
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
pd.set_option('display.max_rows', None) # to display all rows
pd.set_option('display.max_colwidth', None) # to display full text in columns

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Wajdan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wajdan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [110]:
df=pd.read_csv('drugsComTrain_raw.csv')

In [111]:
df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil""",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medications and so far this is the most effective.""",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ingredients are similar. When my other pills ended, I started Lybrel immediately, on my first day of period, as the instructions said. And the period lasted for two weeks. When taking the second pack- same two weeks. And now, with third pack things got even worse- my third period lasted for two weeks and now it&#039;s the end of the third week- I still have daily brown discharge.\r\nThe positive side is that I didn&#039;t have any other side effects. The idea of being period free was so tempting... Alas.""",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth control. I&#039;m glad I went with the patch, I have been on it for 8 months. At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control. Other than that in happy with the patch""",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around. I feel healthier, I&#039;m excelling at my job and I always have money in my pocket and my savings account. I had none of those before Suboxone and spent years abusing oxycontin. My paycheck was already spent by the time I got it and I started resorting to scheming and stealing to fund my addiction. All that is history. If you&#039;re ready to stop, there&#039;s a good chance that suboxone will put you on the path of great life again. I have found the side-effects to be minimal compared to oxycontin. I&#039;m actually sleeping better. Slight constipation is about it for me. It truly is amazing. The cost pales in comparison to what I spent on oxycontin.""",9,27-Nov-16,37


In [112]:
df.value_counts('condition')

condition
Birth Control                                                          28788
Depression                                                              9069
Pain                                                                    6145
Anxiety                                                                 5904
Acne                                                                    5588
Bipolar Disorde                                                         4224
Insomnia                                                                3673
Weight Loss                                                             3609
Obesity                                                                 3568
ADHD                                                                    3383
Diabetes, Type 2                                                        2554
Emergency Contraception                                                 2463
High Blood Pressure                                               

In [113]:
# dropping condition with less than 100 value_counts
df=df[df.groupby('condition').condition.transform('count').ge(1000)]

In [114]:
df.shape

(110121, 7)

In [115]:
#dropping all the columns except for review and condition
df.drop(['uniqueID','drugName','rating','date','usefulCount'],axis=1,inplace=True)

In [116]:
df.value_counts('condition')

condition
Birth Control                  28788
Depression                      9069
Pain                            6145
Anxiety                         5904
Acne                            5588
Bipolar Disorde                 4224
Insomnia                        3673
Weight Loss                     3609
Obesity                         3568
ADHD                            3383
Diabetes, Type 2                2554
Emergency Contraception         2463
High Blood Pressure             2321
Vaginal Yeast Infection         2274
Abnormal Uterine Bleeding       2096
Bowel Preparation               1859
ibromyalgia                     1791
Smoking Cessation               1780
Migraine                        1694
Anxiety and Stress              1663
Major Depressive Disorde        1607
Constipation                    1595
Panic Disorde                   1463
Chronic Pain                    1455
Migraine Prevention             1413
Urinary Tract Infection         1316
Muscle Spasm                

In [117]:
df.head()

,condition,review
1,ADHD,"""My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medications and so far this is the most effective."""
2,Birth Control,"""I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ingredients are similar. When my other pills ended, I started Lybrel immediately, on my first day of period, as the instructions said. And the period lasted for two weeks. When taking the second pack- same two weeks. And now, with third pack things got even worse- my third period lasted for two weeks and now it&#039;s the end of the third week- I still have daily brown discharge.\r\nThe positive side is that I didn&#039;t have any other side effects. The idea of being period free was so tempting... Alas."""
3,Birth Control,"""This is my first time using any form of birth control. I&#039;m glad I went with the patch, I have been on it for 8 months. At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control. Other than that in happy with the patch"""
4,Opiate Dependence,"""Suboxone has completely turned my life around. I feel healthier, I&#039;m excelling at my job and I always have money in my pocket and my savings account. I had none of those before Suboxone and spent years abusing oxycontin. My paycheck was already spent by the time I got it and I started resorting to scheming and stealing to fund my addiction. All that is history. If you&#039;re ready to stop, there&#039;s a good chance that suboxone will put you on the path of great life again. I have found the side-effects to be minimal compared to oxycontin. I&#039;m actually sleeping better. Slight constipation is about it for me. It truly is amazing. The cost pales in comparison to what I spent on oxycontin."""
6,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I took the Plan B 26 hours later, and took a pregnancy test two weeks later - - I&#039;m pregnant."""


In [118]:
for i, col in enumerate(df.columns):
    df.iloc[:,i] = df.iloc[:,i].str.replace('"', '')
    df.iloc[:,i] = df.iloc[:,i].str.replace("&#039;", "'")
    df.iloc[:,i] = df.iloc[:,i].str.replace("&amp;", "and")
    df.iloc[:,i] = df.iloc[:,i].str.replace("\r\n", " ")

In [119]:
df.head()

,condition,review
1,ADHD,"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. We have tried many different medications and so far this is the most effective."
2,Birth Control,"I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ingredients are similar. When my other pills ended, I started Lybrel immediately, on my first day of period, as the instructions said. And the period lasted for two weeks. When taking the second pack- same two weeks. And now, with third pack things got even worse- my third period lasted for two weeks and now it's the end of the third week- I still have daily brown discharge. The positive side is that I didn't have any other side effects. The idea of being period free was so tempting... Alas."
3,Birth Control,"This is my first time using any form of birth control. I'm glad I went with the patch, I have been on it for 8 months. At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control. Other than that in happy with the patch"
4,Opiate Dependence,"Suboxone has completely turned my life around. I feel healthier, I'm excelling at my job and I always have money in my pocket and my savings account. I had none of those before Suboxone and spent years abusing oxycontin. My paycheck was already spent by the time I got it and I started resorting to scheming and stealing to fund my addiction. All that is history. If you're ready to stop, there's a good chance that suboxone will put you on the path of great life again. I have found the side-effects to be minimal compared to oxycontin. I'm actually sleeping better. Slight constipation is about it for me. It truly is amazing. The cost pales in comparison to what I spent on oxycontin."
6,Emergency Contraception,"He pulled out, but he cummed a bit in me. I took the Plan B 26 hours later, and took a pregnancy test two weeks later - - I'm pregnant."


In [120]:
stop = stopwords.words('english')

In [121]:
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [122]:
lemmatizer=WordNetLemmatizer()
def review_to_words(raw_review):
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text() # to remove html tags
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)# to remove non letters
    words = letters_only.lower().split()# to convert to lower case and split
    words = [w for w in words if not w in stop]# to remove stopwords
    words = [lemmatizer.lemmatize(w) for w in words]# to lemmatize
    return(" ".join(words))

In [123]:
df['review']=df['review'].apply(review_to_words)

C:\Users\Wajdan\AppData\Local\Temp\ipykernel_15356\33896917.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'html.parser').get_text() # to remove html tags


In [124]:
df.head()

,condition,review
1,ADHD,son halfway fourth week intuniv became concerned began last week started taking highest dose two day could hardly get bed cranky slept nearly hour drive home school vacation unusual called doctor monday morning said stick day see school getting morning last two day problem free much agreeable ever le emotional good thing le cranky remembering thing overall behavior better tried many different medication far effective
2,Birth Control,used take another oral contraceptive pill cycle happy light period max day side effect contained hormone gestodene available u switched lybrel ingredient similar pill ended started lybrel immediately first day period instruction said period lasted two week taking second pack two week third pack thing got even worse third period lasted two week end third week still daily brown discharge positive side side effect idea period free tempting ala
3,Birth Control,first time using form birth control glad went patch month first decreased libido subsided downside made period longer day exact used period day max also made cramp intense first two day period never cramp using birth control happy patch
4,Opiate Dependence,suboxone completely turned life around feel healthier excelling job always money pocket saving account none suboxone spent year abusing oxycontin paycheck already spent time got started resorting scheming stealing fund addiction history ready stop good chance suboxone put path great life found side effect minimal compared oxycontin actually sleeping better slight constipation truly amazing cost pale comparison spent oxycontin
6,Emergency Contraception,pulled cummed bit took plan b hour later took pregnancy test two week later pregnant


In [125]:
x=df['review']
y=df['condition']

In [126]:
x_train,x_test,y_train,y_test=train_test_split(x,y,stratify=y,test_size=0.2,random_state=3)

In [127]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    # plt.figure(figsize=(20,20))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title,fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        # cm=np.round(cm,2)
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i,cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [128]:
# bag of words
count_vectorizer=CountVectorizer(stop_words='english')
count_train=count_vectorizer.fit_transform(x_train)
count_test=count_vectorizer.transform(x_test)

In [129]:
# machine learning model : Multinomial Naive Bayes
mnb=MultinomialNB()
mnb.fit(count_train,y_train)
pred=mnb.predict(count_test)
score=metrics.accuracy_score(y_test,pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.759


In [130]:
# machone learning model : Passive Aggressive Classifier
pac=PassiveAggressiveClassifier()
pac.fit(count_train,y_train)
pred=pac.predict(count_test)
score=metrics.accuracy_score(y_test,pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.789


In [131]:
# tfidf
tfidf_vectorizer=TfidfVectorizer(stop_words='english',max_df=0.7)
tfidf_train=tfidf_vectorizer.fit_transform(x_train)
tfidf_test=tfidf_vectorizer.transform(x_test)
# naive bayes
mnb=MultinomialNB()
mnb.fit(tfidf_train,y_train)
pred=mnb.predict(tfidf_test)
score=metrics.accuracy_score(y_test,pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.537


In [132]:
# tfid passive aggressive classifier
tfidf_vectorizer=TfidfVectorizer(stop_words='english',max_df=0.7)
tfidf_train=tfidf_vectorizer.fit_transform(x_train)
tfidf_test=tfidf_vectorizer.transform(x_test)

pac=PassiveAggressiveClassifier()
pac.fit(tfidf_train,y_train)
pred=pac.predict(tfidf_test)
score=metrics.accuracy_score(y_test,pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.825


In [133]:
# TFIDf bigram
tfidf_vectorizer=TfidfVectorizer(stop_words='english',max_df=0.7,ngram_range=(1,2))
tfidf_train=tfidf_vectorizer.fit_transform(x_train)
tfidf_test=tfidf_vectorizer.transform(x_test)

In [134]:
# passive aggressive classifier
pass_tf=PassiveAggressiveClassifier()
pass_tf.fit(tfidf_train,y_train)
pred=pass_tf.predict(tfidf_test)
score=metrics.accuracy_score(y_test,pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.872


In [135]:
#trigram passive aggressive classifier
tfidf_vectorizer=TfidfVectorizer(stop_words='english',max_df=0.7,ngram_range=(1,3))
tfidf_train=tfidf_vectorizer.fit_transform(x_train)
tfidf_test=tfidf_vectorizer.transform(x_test)

# passive aggressive classifier
pass_tf=PassiveAggressiveClassifier()
pass_tf.fit(tfidf_train,y_train)
pred=pass_tf.predict(tfidf_test)
score=metrics.accuracy_score(y_test,pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.874


In [136]:
#5gram passive aggressive classifier
tfidf_vectorizer=TfidfVectorizer(stop_words='english',max_df=0.7,ngram_range=(1,5))
tfidf_train=tfidf_vectorizer.fit_transform(x_train)
tfidf_test=tfidf_vectorizer.transform(x_test)

# passive aggressive classifier
pass_tf=PassiveAggressiveClassifier()
pass_tf.fit(tfidf_train,y_train)
pred=pass_tf.predict(tfidf_test)
score=metrics.accuracy_score(y_test,pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.874


In [137]:
# save model
import pickle
pickle.dump(pass_tf,open('pac_model.pkl','wb'))


In [138]:
# load model
pac_model=pickle.load(open('pac_model.pkl','rb'))

In [139]:
df_test=pd.read_csv('drugsComTest_raw.csv')
df_test.value_counts('condition')

condition
Birth Control                                                     9648
Depression                                                        3095
Pain                                                              2100
Anxiety                                                           1908
Acne                                                              1847
Bipolar Disorde                                                   1380
Weight Loss                                                       1248
Insomnia                                                          1231
Obesity                                                           1189
ADHD                                                              1126
Emergency Contraception                                            827
Vaginal Yeast Infection                                            811
Diabetes, Type 2                                                   808
High Blood Pressure                                                

In [142]:
df_test=df_test[df_test.groupby('condition').condition.transform('count').ge(324)]

In [143]:
df_test.drop(['uniqueID','drugName','rating','date','usefulCount'],axis=1,inplace=True)

In [144]:
for i, col in enumerate(df_test.columns):
    df_test.iloc[:,i] = df_test.iloc[:,i].str.replace('"', '')
    df_test.iloc[:,i] = df_test.iloc[:,i].str.replace("&#039;", "'")
    df_test.iloc[:,i] = df_test.iloc[:,i].str.replace("&amp;", "and")
    df_test.iloc[:,i] = df_test.iloc[:,i].str.replace("\r\n", " ")

In [145]:
stop = stopwords.words('english')

In [146]:
df_test['review']=df_test['review'].apply(review_to_words)

C:\Users\Wajdan\AppData\Local\Temp\ipykernel_15356\33896917.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'html.parser').get_text() # to remove html tags


In [152]:
text=df_test['review']
test=tfidf_vectorizer.transform(text)
pred=pac_model.predict(test)
df_test['predicted condition']=pred
# drop pred col
df_test.drop('pred',axis=1,inplace=True)

In [155]:
# showing rows where predicted condition is equal to condition
df_test[df_test['predicted condition']==df_test['condition']].head(20)

,condition,review,predicted condition
0,Depression,tried antidepressant year citalopram fluoxetine amitriptyline none helped depression insomnia anxiety doctor suggested changed onto mg mirtazapine medicine saved life thankfully side effect especially common weight gain actually lost alot weight still suicidal thought mirtazapine saved,Depression
2,Urinary Tract Infection,quick reduction symptom,Urinary Tract Infection
3,Weight Loss,contrave combine drug used alcohol smoking opioid cessation people lose weight also help control eating doubt obesity caused sugar carb addiction powerful drug taking five day good news seems go work immediately feel hungry want food really care eat fill stomach since day know lost weight scale clothes feel little looser maybe pound two hoping month medication develop healthier habit continue without aid contrave,Weight Loss
4,Birth Control,birth control one cycle reading review type similar birth control bit apprehensive start im giving birth control long enough far love birth control side effect minimal like im even birth control experienced mild headache nausea ive feeling great got period cue third day inactive pill idea coming zero pm period light barely cramping unprotected sex first month obviously get pregnant pleased highly recommend,Birth Control
6,Birth Control,copper coil month really excited thought taking hormone good pain however nearly fainted insertion belive painful doctor say painful well month period last day pain day random twang especially left side considering whether want put intense pain heavy period recommend somebody already heavy painful period right,Birth Control
7,Migraine Prevention,great week last week headache went away tylenol chronic daily headache go away matter took still little sleepy day know get better take mg night,Migraine Prevention
9,Birth Control,pill almost two year work far getting pregnant however experience first make huge difference month sex drive went along dry moodiness increased drastically would cry one second get angry husband anything everything skin gotten lot worse broke place never last week yaz,Birth Control
11,Vaginal Yeast Infection,honestly day one day treatment yes burn bit leak dont lay insertion im faithful work,Vaginal Yeast Infection
12,Weight Loss,waste money curb appetite make feel full,Weight Loss
14,Insomnia,ditto rebound sleepless discontinued done strange thing memory including taking additional ambien helped sleep extreme stress watch trying learn sleep naturally,Insomnia


In [157]:
# showing the rows where predicted condition is not equal to condition
df_test[df_test['predicted condition']!=df_test['condition']].head(20)

,condition,review,predicted condition
17,Acne,ginanvi month switched pill due high cost ginavi insurance ginanvi cleared acne short time used hoping birth control would mess wrong month using pill skin went crazy horrible breakout month later still horrible acne want pill help acne also made period weird would little period one month heavy one next month heavier flow boob would hurt lot also noticed beginning taking would moody extremely irritable time time switch,Birth Control
18,Bipolar Disorde,medication helped sleep eventually became ineffective sleep aid also help calm severe stress anxiety panic,Insomnia
19,Urinary Tract Infection,dose ciprofloxacn felt better,Depression
22,Bipolar Disorde,day kicked every noise house would make jump picture tv would feel make jump literally felt like skin crawling moving melding thing blanket mind focused keyed every piece white noise sweaty night hot flash chill,ADHD
26,Panic Disorde,klonopin definitely given life back,Anxiety
41,Bipolar Disorde,started taking gabapentin experimentally treat chronic depression prescribed mg day dos mg experienced significant improvement impulse control depression general went shut going walk enjoying time company others lower dose mg day experience much improvement save decrease depressive symptom looking forward larger dose doctor okay,Depression
48,Anxiety,work,ADHD
71,Major Depressive Disorde,major depression time started menstruating would cycle cycle caused paranoia fear seen outside home feared going mail box yet time would sign thing like avon lady felt crazy mid divorced alcoholic doctor prescribed prozac seem help later recovery still profoundly depressed put zoloft result instantaneous felt like window life opened afraid looked forward getting bed found thing liked healthy relationship good job life given life back year sober thank zoloft,Depression
75,Generalized Anxiety Disorde,pregabalin miraculous first dose mg three time day anxiety reversed really thought good true tried prozac lorazepam different time life commonly known fourth fifth trial ssri peter actually become counter productive even case causing serotonin syndrome two year ago stressful marriage break taking benzodiazepine keep lid anxiety little effect gp respected wish try lyrica still two year astounded transformation feel like always felt feel cessation anxiety sense well pervades,Anxiety
84,Pain,beneficial taken muscle relaxer extreme pain neck arm also taking naproxen,Muscle Spasm


In [158]:
# test accuracy
score=metrics.accuracy_score(df_test['condition'],df_test['predicted condition'])
print("accuracy:   %0.3f" % score)

accuracy:   0.871
